# **1. Setting up Dependencies**


## Connect to Drive

In [32]:
!pip install opencv-python


## Download Project

In [1]:
!git clone https://github.com/WongKinYiu/yolov7.git
!cd yolov7
!wget https://raw.githubusercontent.com/WongKinYiu/yolov7/u5/requirements.txt
!pip install -r requirements.txt

Cloning into 'yolov7'...
Updating files:  75% (82/108)
Updating files:  76% (83/108)
Updating files:  77% (84/108)
Updating files:  78% (85/108)
Updating files:  79% (86/108)
Updating files:  80% (87/108)
Updating files:  81% (88/108)
Updating files:  82% (89/108)
Updating files:  83% (90/108)
Updating files:  84% (91/108)
Updating files:  85% (92/108)
Updating files:  86% (93/108)
Updating files:  87% (94/108)
Updating files:  88% (96/108)
Updating files:  89% (97/108)
Updating files:  90% (98/108)
Updating files:  91% (99/108)
Updating files:  92% (100/108)
Updating files:  93% (101/108)
Updating files:  94% (102/108)
Updating files:  95% (103/108)
Updating files:  96% (104/108)
Updating files:  97% (105/108)
Updating files:  98% (106/108)
Updating files:  99% (107/108)
Updating files: 100% (108/108)
Updating files: 100% (108/108), done.
'wget' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Could not open requirements file: [Errno 2] No s

In [ ]:
cd "C:/Users/Ahmme/Downloads/Yolo-20231224T203553Z-001"

In [2]:
import os
import sys
sys.path.append("C:/Users/Ahmme/Downloads/Yolo-20231224T203553Z-001/Yolo/yolov7")

In [3]:
cd "C:/Users/Ahmme/Downloads/Yolo-20231224T203553Z-001/Yolo/yolov7"

C:\Users\Ahmme\Downloads\Yolo-20231224T203553Z-001\Yolo\yolov7


## Getting Yolo Models

## Display Functions

In [4]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random
import tensorflow as tf
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

##Parameters

In [28]:
classes_to_filter = None #You can give list of classes to filter by name ['train','person' ]


opt  = {
    "weights": "weights/yolov7.pt", # Path to weights file default weights are for nano model
    "yaml"   : "data/coco.yaml",
    "img-size": 640, # default image size
    "conf-thres": 0.25, # confidence threshold for inference.
    "iou-thres" : 0.45, # NMS IoU threshold for inference.
    "device" : 'cpu',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : classes_to_filter  # list of classes to filter or None
}

#**2. Running on a single image**

In [43]:
# Give path of source image
source_image_path = "C:/Users/Ahmme/Downloads/https___data.amirite.com_posts_a6_b7_a6369648176e.jpg"


with torch.no_grad():
  weights, imgsz = opt['weights'], opt['img-size']
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)  # load FP32 model
  stride = int(model.stride.max())  # model stride
  imgsz = check_img_size(imgsz, s=stride)  # check img_size
  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

  img0 = cv2.imread(source_image_path)
  img = letterbox(img0, imgsz, stride=stride)[0]
  img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
  img = np.ascontiguousarray(img)
  img = torch.from_numpy(img).to(device)
  img = img.half() if half else img.float()  # uint8 to fp16/32
  img /= 255.0  # 0 - 255 to 0.0 - 1.0
  if img.ndimension() == 3:
    img = img.unsqueeze(0)

  # Inference
  t1 = time_synchronized()
  pred = model(img, augment= False)[0]

  # Apply NMS
  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:

      classes.append(names.index(class_name))

  if classes:

    classes = [i for i in range(len(names)) if i not in classes]


  pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
  t2 = time_synchronized()
  for i, det in enumerate(pred):
    s = ''
    s += '%gx%g ' % img.shape[2:]  # print string
    gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
    if len(det):
      det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

      for c in det[:, -1].unique():
        n = (det[:, -1] == c).sum()  # detections per class
        s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
        print(s)

      for *xyxy, conf, cls in reversed(det):

        label = f'{names[int(cls)]} {conf:.2f}'
        plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)



YOLOR  v0.1-128-ga207844 torch 2.1.0+cu118 CPU



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
640x640 2 bananas, 
640x640 2 bananas, 5 apples, 
640x640 2 bananas, 5 apples, 2 oranges, 


In [44]:
import cv2
cv2.imshow('Image', img0)
cv2.waitKey(0)
cv2.destroyAllWindows()

#**3.Running YOLOv7 on Video**

# using video

##Enter Video Path

In [47]:
#give the full path to video, your video will be in the Yolov7 folder
video_path = "C:/Users/Ahmme/Downloads/pexels_videos_2016738 (1080p).mp4"

##Run Yolo on Video

In [48]:
# Initializing video object
video = cv2.VideoCapture(video_path)


#Video information
fps = video.get(cv2.CAP_PROP_FPS)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
nframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialzing object for writing video output
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, (w, h))
torch.cuda.empty_cache()
# Initializing model and setting it for inference
with torch.no_grad():
  weights, imgsz = opt['weights'], opt['img-size']
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)  # load FP32 model
  stride = int(model.stride.max())  # model stride
  imgsz = check_img_size(imgsz, s=stride)  # check img_size
  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:

      classes.append(names.index(class_name))

  if classes:

    classes = [i for i in range(len(names)) if i not in classes]

  for j in range(nframes):

      ret, img0 = video.read()

      if ret:
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
          img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment= False)[0]


        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
        t2 = time_synchronized()
        for i, det in enumerate(pred):
          s = ''
          s += '%gx%g ' % img.shape[2:]  # print string
          gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
          if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
              print(s)

            for *xyxy, conf, cls in reversed(det):

              label = f'{names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

        print(f"{j+1}/{nframes} frames processed")
        output.write(img0)
      else:
        break
output.release()
video.release()

YOLOR  v0.1-128-ga207844 torch 2.1.0+cu118 CPU



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
384x640 1 apple, 
384x640 1 apple, 22 oranges, 
1/228 frames processed
384x640 1 apple, 
384x640 1 apple, 21 oranges, 
2/228 frames processed
384x640 2 apples, 
384x640 2 apples, 19 oranges, 
3/228 frames processed
384x640 3 apples, 
384x640 3 apples, 20 oranges, 
4/228 frames processed
384x640 2 apples, 
384x640 2 apples, 18 oranges, 
5/228 frames processed
384x640 3 apples, 
384x640 3 apples, 17 oranges, 
6/228 frames processed
384x640 2 apples, 
384x640 2 apples, 20 oranges, 
7/228 frames processed
384x640 2 apples, 
384x640 2 apples, 17 oranges, 
8/228 frames processed
384x640 2 apples, 
384x640 2 apples, 19 oranges, 
9/228 frames processed
384x640 2 apples, 
384x640 2 apples, 17 oranges, 
10/228 frames processed
384x640 2 apples, 
384x640 2 apples, 19 oranges, 
11/228 frames processed
384x640 1 person, 
384x640 1 person, 4 apples, 
384x640 1 person, 4 apples, 18 oranges, 
12/228 frames processed
384x640 

In [49]:
from IPython.display import FileLink

# ... (Previous code remains unchanged)

# Release the output video capture object
output.release()
video.release()

# Reopen the output video file for display
output_video = cv2.VideoCapture('output.mp4')

# Define the codec and create a VideoWriter object for saving the displayed video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_display = cv2.VideoWriter('output_display.avi', fourcc, fps, (w, h))

while True:
    ret, frame = output_video.read()
    if not ret:
        break

    # Display the frame
    cv2.imshow('Output Video', frame)

    # Write the frame to the new video file
    output_display.write(frame)

    # Break the loop if the user presses 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the output video capture and display objects
output_video.release()
output_display.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

# Create a download link for the displayed video file
display_link = FileLink('output_display.avi')
display_link


C:\Users\Ahmme\Downloads\Yolo-20231224T203553Z-001\Yolo\yolov7\output_display.avi

# using camera

In [46]:
import cv2
import torch
from utils.general import non_max_suppression, scale_coords
from utils.torch_utils import select_device
# Initializing video object for camera input
video = cv2.VideoCapture(0)  # 0 corresponds to the default camera

# Video information (adjust as needed)
fps = 30  # Adjust the frame per second according to your camera
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
nframes = 0  # For camera input, set the number of frames to 0
torch.cuda.empty_cache()
# Initializing model and setting it for inference
with torch.no_grad():
  weights, imgsz = opt['weights'], opt['img-size']
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)  # load FP32 model
  stride = int(model.stride.max())  # model stride
  imgsz = check_img_size(imgsz, s=stride)  # check img_size
  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:

      classes.append(names.index(class_name))

  if classes:

    classes = [i for i in range(len(names)) if i not in classes]

  for j in range(nframes):

      ret, img0 = video.read()

      if ret:
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
          img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment= False)[0]


        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
        t2 = time_synchronized()
        for i, det in enumerate(pred):
          s = ''
          s += '%gx%g ' % img.shape[2:]  # print string
          gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
          if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            for *xyxy, conf, cls in reversed(det):

              label = f'{names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

        print(f"{j+1}/{nframes} frames processed")
        output.write(img0)
      else:
        break
while True:
    ret, img0 = video.read()

    if ret:
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes=classes, agnostic=False)
        t2 = time_synchronized()

        for i, det in enumerate(pred):
            s = ''
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                    print(s)

                for *xyxy, conf, cls in reversed(det):
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

        cv2.imshow('Result', img0)

        # Write the frame to the output video file
        output.write(img0)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break

# Release the video capture and video writer objects
video.release()
output.release()

# Destroy all OpenCV windows
cv2.destroyAllWindows()

YOLOR  v0.1-128-ga207844 torch 2.1.0+cu118 CPU



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 
480x640 1 person, 2 apples, 
480x640 1 person, 
480x640 1 person, 1 sports ball, 
480x640 1 person, 1 sports ball, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 sports ball, 
480x640 1 person, 
480x640 1 person, 1 sports ball, 
480x640 1 person, 
480x640 1 person, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 apple, 
480x640 1 person, 1 apple, 1 hot dog, 
480x640 1 person, 
480x640 1 person, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 apple, 
480x640 2 persons, 
480x640 2 persons, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 sports ball, 
480x640 1 person, 
480x640 1 person, 1 banana, 
480x640 1 person, 1 banana, 1 apple, 
480x640 1 person, 
480x640 1 person, 1 sports ball, 
480x640 1 person, 
480x640 1 person, 1 appl